In [4]:
import pymongo 
from datetime import datetime
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint as pp

In [5]:
# connect to database
connection = MongoClient('localhost', 27017)

db = connection.sample


In [6]:
# handle to names collection
towns = db.towns
towns.drop()

In [7]:
town = {
"name": "New York",
"population": 22200000,
"last_census": datetime(2007, 7, 31),
"famous_for": [ "statue of liberty", "food" ],
"mayor" : {
    "name" : "Michael Bloomberg",
    "party" : "I"
    }
}

In [8]:
towns.insert_one(town)

In [9]:
for town in towns.find():
    pp(town)

{u'_id': ObjectId('5620fe10801d3208c0d3ce50'),
 u'famous_for': [u'statue of liberty', u'food'],
 u'last_census': datetime.datetime(2007, 7, 31, 0, 0),
 u'mayor': {u'name': u'Michael Bloomberg', u'party': u'I'},
 u'name': u'New York',
 u'population': 22200000}


In [10]:
def insertCity(name, population, last_census, famous_for, mayor_info):
    town = {
        "name": name,
        "population": population,
        "last_census": datetime.strptime(last_census, '%Y-%d-%m'),
        "famous_for": famous_for,
        "mayor" : mayor_info
        }
    return str(towns.insert_one(town).inserted_id)

In [11]:
insertCity("Punxsutawney", 6200, '2008-31-01',
            ["phil the groundhog"], { "name" : "Jim Wehrle" }
)

'5620fe12801d3208c0d3ce51'

In [12]:
town_id = insertCity("Portland", 582000, '2007-20-09',
["beer", "food"], { "name" : "Sam Adams", "party" : "D" }
)

In [13]:
for town in db.towns.find({ "_id" : ObjectId(town_id) }):
    pp(town)

{u'_id': ObjectId('5620fe12801d3208c0d3ce52'),
 u'famous_for': [u'beer', u'food'],
 u'last_census': datetime.datetime(2007, 9, 20, 0, 0),
 u'mayor': {u'name': u'Sam Adams', u'party': u'D'},
 u'name': u'Portland',
 u'population': 582000}


In [14]:
for town in db.towns.find( {"_id" : ObjectId(town_id) }, { "name" : 1, "_id" : 0 }):
    pp(town)

{u'name': u'Portland'}


In [15]:
for town in db.towns.find( {"_id" : ObjectId(town_id) }, { "name" : 0 }):
    pp(town)

{u'_id': ObjectId('5620fe12801d3208c0d3ce52'),
 u'famous_for': [u'beer', u'food'],
 u'last_census': datetime.datetime(2007, 9, 20, 0, 0),
 u'mayor': {u'name': u'Sam Adams', u'party': u'D'},
 u'population': 582000}


In [40]:
for town in db.towns.find({},
{ "name" : 1, "population" : 1, "last_census" : 1, "_id" : 0}):
    pp(town)

{u'last_census': datetime.datetime(2007, 7, 31, 0, 0),
 u'name': u'New York',
 u'population': 22200000}
{u'last_census': datetime.datetime(2008, 1, 31, 0, 0),
 u'name': u'Punxsutawney',
 u'population': 6200}
{u'last_census': datetime.datetime(2007, 9, 20, 0, 0),
 u'name': u'Portland',
 u'population': 582000}


In [32]:
import re

db.towns.find_one(
{ "name" : re.compile("^P") ,"population" : { "$lt" : 10000 }},
{ "name" : 1, "population" : 1 , "_id" : 0}
)

{u'name': u'Punxsutawney', u'population': 6200}

In [29]:
regx = re.compile("^P", re.IGNORECASE)
db.towns.find_one({"name": regx})

{u'_id': ObjectId('5620fe12801d3208c0d3ce51'),
 u'famous_for': [u'phil the groundhog'],
 u'last_census': datetime.datetime(2008, 1, 31, 0, 0),
 u'mayor': {u'name': u'Jim Wehrle'},
 u'name': u'Punxsutawney',
 u'population': 6200}

In [38]:
population_range = {}
population_range['$lt'] = 1000000
population_range['$gt'] = 10000

for t in db.towns.find(
{ "name" : re.compile("^P"), "population" : population_range },
{ "name": 1 }
): pp(t)

{u'_id': ObjectId('5620fe12801d3208c0d3ce52'), u'name': u'Portland'}


In [45]:
for t in db.towns.find(
{ "last_census" : { "$lte" : datetime.strptime('2008-01-01', '%Y-%d-%m') } },
{ "_id" : 0, "name": 1 , "last_census" : 1}
): pp(t)

{u'last_census': datetime.datetime(2007, 7, 31, 0, 0), u'name': u'New York'}
{u'last_census': datetime.datetime(2007, 9, 20, 0, 0), u'name': u'Portland'}
